---
title: "Integrated Dynamic N-mixture Model with NIMBLE"
format: html
theme:
  - cosmo
---

## Process sub-model:
The process sub-model specifies how abundance varies across space and time. More specifically, the process sub- model follows the model described by Hostetler and Chandler (2015) such that,

$$
N_{i,t=1} \sim Poisson(\lambda_{i,t})
$$
$$
\underbrace{log(\lambda_{i,t=1})}_\text{log abundance at time $t=1$} = 
\underbrace{\beta^{[0]} + \beta^{[0]}_1 \times x_{i,1,1} + ... + \beta^{[0]}_k \times x_{i,1,k}}_\text{linear predictor of state process at time $t=1$}
$$
$$
\underbrace{\lambda_{i,t≥2}}_\text{expected abundance at time $t≥2$} = 
\underbrace{N_{i,t-1} \times \rho_{i,t-1} + \delta}_\text{linear predictor of previous abundance, population growth, and immigration}
$$
$$
\underbrace{log(\rho_{i,t-1})}_\text{log population growth rate} = 
\underbrace{\beta^{[\rho]} + \beta^{[\rho]}_1 \times N_{i,t-1} + \beta^{[\rho]}_1 \times x_{i,1,1} + ... + \beta^{[\rho]}_k \times x_{i,1,k}}_\text{linear predictor of population growth rate}
$$

in which $N_{i,t}$ is the true abundance at site $i$ in year $t$ that follows a Poisson distribution, and $\lambda_{i,t}$ is its expectation. In the first year, $\lambda_{i,t=1}$ is a function of $k$ environmental covariates $x$'s with intercept and slope parameters $\beta^{[0]}$. For subsequent years (i.e. $t≥2$), $\lambda_{i,t≥2}$ is calculated from population size in the previous year $N_{i,t−1}$, population growth rate $\rho_{i,t-1}$, and expected number of immigrants $\delta$. Population growth rate $\rho_{i,t-1}$ is further a function of density and environmental covariates with intercept and slope parameters \beta^{[\rho]}.

The code for this submodel is as follows: 

```{default}
# Process model
# for (i in 1:nsite) {
#     z[i] ~ dbinom(zeta, 1)
#     lambda0[i] <- exp(
#         beta_lambda0[1] + 
#         beta_lambda0[2] * gra[i,1] + 
#         beta_lambda0[3] * tem[i,1] + 
#         beta_lambda0[4] * tem[i,1] * tem[i,1])
#     N[i,1] ~ dpois(lambda0[i] * z[i] + 5e-3 * (1 - z[i])) 

#     for (t in 2:nyear) {
#         rho[i,t-1] <- exp(
#             beta_rho[1] + 
#             beta_rho[2] * gra[i,t] + 
#             beta_rho[3] * tem[i,t] + 
#             beta_rho[4] * tem[i,t] * tem[i,t])
#         N[i,t] ~ dpois(N[i,t-1] * rho[i,t-1] * z[i] + 5e-3 * (1 - z[i]))
#     } # t
# } # i
```


::: {.callout-important}
Alex, figure out the code bit that includes a parameter multiplied by the compliment of $z_[i]$: 
```{default}
# N[i,1] ~ dpois(lambda0[i] * z[i] + 5e-3 * (1 - z[i]))
```
Is this the scaling parameter?
:::

## Observation sub-model:

### Rigorously Sampled data (IMBCR): 
The first observation sub-model links RS data with the true abundance while utilizing distance and removal sampling information (Amundson et al., 2014; Royle et al., 2004; Seber, 1982) such that...
$$
y_{i,t,j,k,s} \sim Multinomial(N_{i, t}, \pi_{i, t, j, k, s})
$$
$$
\pi_{i, t, j, k, s}=\frac{2 \times \sigma^2_{i,t,j}\times [exp(\frac{-D^2_{k,L}}{2 \times \sigma^2_{i,t,j}}) - exp(\frac{-D^2_{k,U}}{2 \times \sigma^2_{i,t,j}})]}{r^2} \times [(1-\theta_{i,t,j}^{s-1}) \times \theta_{i,t,j}]
$$
$$
log(\sigma_{i,t,j})=\alpha^{[\sigma]}_0 + \alpha^{[\sigma]}_1 \times w_{i,t,j,1} + ... + \alpha^{[\sigma]}_l \times w_{i,t,j,l}
$$
$$
log(\theta_{i,t,j})=\alpha^{[\theta]}_0 + \alpha^{[\theta]}_1 \times w_{i,t,j,1} + ... + \alpha^{[\theta]}_l \times w_{i,t,j,l}
$$

in which $y_{i,t,j,k,s}$ is the RS count at site $i$ in year $t$ at point $j$ in distance bin $k$ and time interval $s$, and is assumed to follow a multinomial distribution with detection probability $\pi_{i, t, j, k, s}$, $\sigma^2_{i,t,j}$ is the standard deviation for a half-normal distance sampling function describing the decline in perceptibility with distance from the surveyor, and $D^2_{k,L}$ and $D^2_{k,U}$ are the distances of the lower and upper bounds of the $k$th distance bin, respectively, $r$ is the radius of the area within which observations are conducted and $\theta_{i,t,j}$ is availability within one time interval. Further, $\sigma^2_{i,t,j}$ and $\theta_{i,t,j}$ are functions of the same RS-specific covariates $w$'s with intercept and slope parameters $\alpha^{[\sigma]}$ and $\alpha^{[\theta]}$, respectively, to capture potential heterogeneity in perceptibility and availability across sites, years, and points. Further, the mean $\sigma^2_{i,t,j}$ and $\theta^2_{i,t,j}$ can be expressed as and $\bar{\sigma}=exp(\alpha^{[\sigma]}_0)$ and $\bar{\theta} = logit^{-1}(\alpha^{[\theta]}_0)$, respectively, since $w$'s are standardized to have 0 mean.

The Nimble code for the RS observation submodel is as follows:
```{default}
#   for (d in 1:ndist) {
#     intval[d] <- sigma^2 * (exp(-1*(breaks[d]^2)/(2*sigma^2)) - exp(-1*(breaks[d+1]^2)/(2*sigma^2)))
#     psi[d] <- 2 * intval[d] / (cutoff ^ 2)
#   } # d
#   psi_sum <- sum(psi[1:ndist])
#   for(d in 1:ndist) {
#     psi_prop[d] <- psi[d] / psi_sum
#   } # d

#   for (r in 1:ntime) {
#     phi[r] <- (1 - theta) ^ (r - 1) * theta
#   } # r
#   phi_sum <- sum(phi[1:ntime])
#   for (r in 1:ntime) {
#     phi_prop[r] <- phi[r] / phi_sum
#   } # r

#   for (d in 1:ndist) {
#     for (r in 1:ntime) {
#       pi[(r-1)*ndist+d] <- psi_prop[d] * phi_prop[r]
#     } # r
#   } # d

#   for(k in 1:nobs_imbcr) {
#     imbcr_cnt_sum[k] ~ dbinom(psi_sum * phi_sum, N[sites_imbcr[k], years_imbcr[k]])
#     imbcr_cnt[k,1:(ndist*ntime)] ~ dmultinom(pi[1:(ndist*ntime)], imbcr_cnt_sum[k])
#   } # k
```

### Opportunistically Sampled data (BBS): 
The second observation sub-model links PS1 data with abundance while considering potential sampling biases and observation errors represented by random and fixed effects such that
$$
c_{i,t,j,o} \sim Poisson(N_{i,t} \times \chi_{i,t,j,o})
$$
$$
log(c_{i,t,j,o}) = \alpha^{[\chi]}_0 + \tau + \alpha^{[\chi]}_1 + z_{i,t,j,1} + ... + \alpha^{[\chi]}_1 + z_{i,t,j,l}
$$
where is the PS1 count at site $i$ in year $t$ at point $j$ of cluster (e.g. observations collected by the same observer) o, is a scaling parameter that represents potential sampling biases and observation errors with mean , a cluster-specific random effect with mean 0 and standard deviation and fixed effects of PS1-specific covariates z's with slope parameters.

### Opportunistically Sampled data (BBS): 
The third observation sub-model links PS2 data with abundance while considering potential sampling biases and observation errors represented by fixed effects such that...
$$
e_{i,t,j,o} \sim Poisson(N_{i,t} \times w_{i,t,j,o})
$$
$$
log(w_{i,t,j,o}) = \alpha^{[w]}_0 + \alpha^{[w]}_1 + v_{i,t,j,1} + ... + \alpha^{[w]}_1 + v_{i,t,j,l}
$$

where is the PS2 count at site $i$ in year $t$ at point $j$, and is a scaling parameter with mean and driven by PS2-specific covariates v's with slope parameters . By specifying separate observation sub-models for PS1 and PS2 data, our approach enables estimating separate scaling parameters for each data set to achieve flexibility.

```{default}
#   for (k in 1:nroute_bbs) {
#     chi_route_epsilon[k] ~ dnorm(0, sd=chi_route_sd)
#   } # k
#   for (k in 1:nobser_bbs) {
#     chi_obser_epsilon[k] ~ dnorm(0, sd=chi_obser_sd)
#   } # k
#   for (j in 1:nstop_bbs) {
#     chi_stop_sd[j] <- exp(chi_stop_beta[1] + chi_stop_beta[2] * stops_bbs[j])
#   } # j
#   for (k in 1:nobs_bbs) {
#     log_chi_mu[k] <- 
#       chi_mu + 
#       chi_route_epsilon[route_bbs[k]] + 
#       chi_obser_epsilon[obser_bbs[k]] + 
#       chi_new * new_bbs[k]
#     for (j in 1:nstop_bbs) {
#       log_chi[k,j] ~ dnorm(log_chi_mu[k], chi_stop_sd[j])
#       chi[k,j] <- exp(log_chi[k,j])
#       bbs_cnt[k,j] ~ dpois(N[sites_bbs[k], years_bbs[k]] * chi[k,j])
#     } # j
#   } # k
```

### Opportunistically Sampled data (eBird): 
```{default}
#   for (k in 1:nlocat_ebird) {
#     omega_locat_epsilon[k] ~ dnorm(0, sd=omega_locat_sd)
#   } # k
#   for (k in 1:nobser_ebird) {
#     omega_obser_epsilon[k] ~ dnorm(0, sd=omega_obser_sd)
#   } # k
#   for (k in 1:nobs_ebird) {
#     omega_z[k] ~ dbinom(omega_zeta, 1)
#     omega[k] <- exp(
#       omega_beta[1] + 
#       omega_beta[2] * type_ebird[k] + 
#       omega_beta[3] * duration_ebird[k] + 
#       omega_beta[4] * distance_ebird[k] + 
#       omega_beta[5] * n_obsver_ebird[k] + 
#       omega_locat_epsilon[locat_ebird[k]] + 
#       omega_obser_epsilon[obser_ebird[k]])
#     ebird_cnt[k] ~ dpois(N[sites_ebird[k], years_ebird[k]] * omega[k] * omega_z[k] + omega_delta * (1 - omega_z[k]))
#   } # i
```

## Model priors: 
```{default}
#   zeta ~ dunif(0, 1)
#   for (k in 1:4) {
#     beta_lambda0[k] ~ dnorm(0, sd=10)
#     beta_rho[k] ~ dnorm(0, sd=10)
#   } # k
#   sigma ~ dgamma(.01, .01)
#   theta ~ dunif(0, 1)
#   chi_mu ~ dnorm(0, sd=10)
#   chi_route_sd ~ dgamma(.01, .01)
#   chi_obser_sd ~ dgamma(.01, .01)
#   chi_new ~ dnorm(0, sd=10)
#   chi_stop_beta[1] ~ dnorm(0, sd=10)
#   chi_stop_beta[2] ~ T(dnorm(0, sd=10), 0,)
#   for (k in 1:5) {
#     omega_beta[k] ~ dnorm(0, sd=10)
#   } # k
#   omega_locat_sd ~ dgamma(.01, .01)
#   omega_obser_sd ~ dgamma(.01, .01)
#   omega_zeta ~ dunif(0, 1)
#   omega_delta ~ dgamma(.01, .01)
```